# Import Necessary Libraries

In [1]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import random
import math
import collections
import os
import json
import pickle

from sklearn.preprocessing import StandardScaler, MinMaxScaler
from sklearn.neighbors import KNeighborsClassifier
from sklearn.model_selection import GridSearchCV
from sklearn.model_selection import train_test_split
from sklearn.metrics import classification_report, confusion_matrix
from sklearn.model_selection import cross_val_predict

import warnings
warnings.filterwarnings('ignore')

# Load Train Test Split

In [9]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 250, 12) 
Testing Samples : (83, 250, 12)
Training Labels : (332,) 
Testing Labels : (83,)


In [10]:
_ , num_timeframes, num_features = X_train.shape

# Reshape data for the model
-----------------
Samples are a 2D matrix which need to be reshaped to 1D to feed into the model

In [11]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (332, 3000) 
Testing Samples : (83, 3000)
Training Labels : (332,) 
Testing Labels : (83,)


# Grid search over KNN model

In [12]:
parameters = {
    "n_neighbors": list(range(3,11,2)),
    "weights": ['uniform','distance'],
    "metric":['euclidean','manhattan','minkowski','chebyshev']
}

grid = GridSearchCV(KNeighborsClassifier(), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.1s
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.
[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.0s remaining:    0.0s


[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.1s
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.1s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.1s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.1s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.1s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=5, weights=uniform, total=   0.1s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=5, weights=uniform, total=   0.1s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] .

[CV]  metric=minkowski, n_neighbors=5, weights=distance, total=   0.1s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.1s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.1s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.1s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.1s
[CV] metric=minkowski, n_neighbors=9, weights=uniform ................
[CV] .

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   12.9s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski',
                                    'chebyshev'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [13]:
params = grid.best_params_
best_estimator = grid.best_estimator_
print(params)
print(best_estimator)

{'metric': 'manhattan', 'n_neighbors': 3, 'weights': 'distance'}
KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')


In [14]:
clf = best_estimator
clf.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

In [15]:
Y_pred = clf.predict(X_test)
print(classification_report(Y_test, Y_pred))
print(confusion_matrix(Y_test,Y_pred))

              precision    recall  f1-score   support

           0       0.60      0.69      0.64        13
           1       0.67      0.67      0.67        15
           2       0.71      0.71      0.71        14
           3       1.00      0.89      0.94         9
           4       0.79      0.75      0.77        20
           5       0.50      0.50      0.50        12

    accuracy                           0.70        83
   macro avg       0.71      0.70      0.71        83
weighted avg       0.71      0.70      0.70        83

[[ 9  2  2  0  0  0]
 [ 3 10  0  0  1  1]
 [ 2  0 10  0  1  1]
 [ 0  0  1  8  0  0]
 [ 0  1  0  0 15  4]
 [ 1  2  1  0  2  6]]


# Train on entire data

In [16]:
X_train = np.load(os.path.join("..", "IPD", "x_train.npy"))
X_test = np.load(os.path.join("..", "IPD", "x_test.npy"))
Y_train = np.load(os.path.join("..", "IPD", "y_train.npy"))
Y_test = np.load(os.path.join("..", "IPD", "y_test.npy"))

X_train = np.concatenate((X_train,X_test), axis=0)
Y_train = np.concatenate((Y_train,Y_test), axis=0)

_ , num_timeframes, num_features = X_train.shape
X_train = X_train.reshape((-1,num_timeframes*num_features))
from scipy.sparse import coo_matrix
X_sparse = coo_matrix(X_train)

from sklearn.utils import shuffle
X_train, X_sparse, Y_train = shuffle(X_train, X_sparse, Y_train)
X_train = X_train.reshape((-1,num_timeframes,num_features))

In [17]:
X_train = X_train.reshape((-1,num_timeframes*num_features))
X_test = X_test.reshape((-1,num_timeframes*num_features))
print("Training samples : {} \nTesting Samples : {}".format(X_train.shape, X_test.shape))
print("Training Labels : {} \nTesting Labels : {}".format(Y_train.shape, Y_test.shape))

Training samples : (415, 3000) 
Testing Samples : (83, 3000)
Training Labels : (415,) 
Testing Labels : (83,)


In [18]:
parameters = {
    "n_neighbors": list(range(3,11,2)),
    "weights": ['uniform','distance'],
    "metric":['euclidean','manhattan','minkowski','chebyshev']
}

grid = GridSearchCV(KNeighborsClassifier(), parameters, cv=3, verbose=2)
grid.fit(X_train, Y_train)

Fitting 3 folds for each of 32 candidates, totalling 96 fits
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................


[Parallel(n_jobs=1)]: Using backend SequentialBackend with 1 concurrent workers.


[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.2s
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................


[Parallel(n_jobs=1)]: Done   1 out of   1 | elapsed:    0.1s remaining:    0.0s


[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.2s
[CV] metric=euclidean, n_neighbors=3, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=3, weights=uniform, total=   0.2s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.2s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.2s
[CV] metric=euclidean, n_neighbors=3, weights=distance ...............
[CV]  metric=euclidean, n_neighbors=3, weights=distance, total=   0.2s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=5, weights=uniform, total=   0.3s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] . metric=euclidean, n_neighbors=5, weights=uniform, total=   0.3s
[CV] metric=euclidean, n_neighbors=5, weights=uniform ................
[CV] .

[CV]  metric=minkowski, n_neighbors=5, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=uniform ................
[CV] . metric=minkowski, n_neighbors=7, weights=uniform, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=7, weights=distance ...............
[CV]  metric=minkowski, n_neighbors=7, weights=distance, total=   0.2s
[CV] metric=minkowski, n_neighbors=9, weights=uniform ................
[CV] .

[Parallel(n_jobs=1)]: Done  96 out of  96 | elapsed:   21.4s finished


GridSearchCV(cv=3, error_score=nan,
             estimator=KNeighborsClassifier(algorithm='auto', leaf_size=30,
                                            metric='minkowski',
                                            metric_params=None, n_jobs=None,
                                            n_neighbors=5, p=2,
                                            weights='uniform'),
             iid='deprecated', n_jobs=None,
             param_grid={'metric': ['euclidean', 'manhattan', 'minkowski',
                                    'chebyshev'],
                         'n_neighbors': [3, 5, 7, 9],
                         'weights': ['uniform', 'distance']},
             pre_dispatch='2*n_jobs', refit=True, return_train_score=False,
             scoring=None, verbose=2)

In [19]:
clf = grid.best_estimator_
clf.fit(X_train,Y_train)

KNeighborsClassifier(algorithm='auto', leaf_size=30, metric='manhattan',
                     metric_params=None, n_jobs=None, n_neighbors=3, p=2,
                     weights='distance')

# Saving Model

In [20]:
pickle.dump(clf, open(os.path.join("..", "IPD", "KNN_model.pkl"), 'wb'))
print("Model Saved")

Model Saved
